In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180944 sha256=1981a5e39302be9a1a7e70202e9526c2e0fd4e154e5c1edf70f74e71bb88fc61
  Stored in directory: /tmp/pip-ephem-wheel-cache-bjnv320e/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip install keybert

import json
import numpy as np
import re 

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import words
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

import spacy
import en_core_web_lg

from sklearn.feature_extraction.text import TfidfVectorizer

import keybert
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
sentence_model = SentenceTransformer('paraphrase-distilroberta-base-v2')
model = KeyBERT(model=sentence_model)
nlp = en_core_web_lg.load()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
def retrieve_data(fileName):
  # Opening JSON file
  f = open(fileName)
  # returns JSON object as a dictionary
  data = json.load(f)
  return data

def extract_review_and_label(data, label):
  labels = [label] * len(data)
  reviews = [d['stopwords_removal_lemmatization'] for d in data]
  return reviews, labels  

In [ ]:
feature_data = retrieve_data("Feature.json")
feature_reviews, feature_labels = extract_review_and_label(feature_data, 0)

user_experience_data = retrieve_data("UserExperience.json")
user_experience_reviews, user_experience_labels = extract_review_and_label(user_experience_data, 1)

rating_data = retrieve_data("Rating.json")
rating_reviews, rating_labels = extract_review_and_label(rating_data, 2)

bug_data = retrieve_data("Bug.json")
bug_reviews, bug_labels = extract_review_and_label(bug_data, 3)

reviews = np.concatenate(([],bug_reviews))
labels = np.concatenate(([],bug_labels))

randomize = np.arange(len(reviews))
np.random.shuffle(randomize)
reviews = reviews[randomize]
labels = labels[randomize]

In [ ]:
def preprocess(review):

  temp = ""
  lst = []
  temp = re.sub('[^a-zA-Z]',' ',review).strip()
  temp = temp.lower()
  return temp

In [ ]:
results = []
for r in reviews:
  results.append(preprocess(r))

In [ ]:
import pandas as pd
df = pd.DataFrame(results, columns=["sentence"])
df["label"] = labels
df.head()

,sentence,label
0,ever since io update dropbox have not work ...,3.0
1,previous version if user be look at picture s...,3.0
2,seriouslywhy do select all functionality no lo...,3.0
3,nice nice app enjoy but kid have more fun wis...,3.0
4,constant crash io crash minute evernote...,3.0


In [ ]:
def extract_keywords(str_):
  keywords = model.extract_keywords(str_, keyphrase_ngram_range=(2,3), use_mmr=True, diversity=0.3, stop_words='english')
  lst = []
  for k,v in keywords:
    for chunk in list(k.split()):
      if len(str(chunk).split()) > 0:
        for el in str(chunk).split():
          lst.append(el) if el not in lst else 1
      else:
        lst.append(chunk) if chunk not in lst else 1
  return " ".join(lst)

In [ ]:
def extract_bigrams(str_):
  keywords = model.extract_keywords(str_, keyphrase_ngram_range=(2,2), use_mmr=True, diversity=0.3, stop_words='english',top_n=10)
  lst = []
  for k,v in keywords:
    lst.append(tuple(k.split()))    
  return lst

In [ ]:
def extract_trigrams(str_):
  keywords = model.extract_keywords(str_, keyphrase_ngram_range=(3,3), use_mmr=True, diversity=0.3, stop_words='english',top_n=5)
  lst = []
  for k,v in keywords:
    lst.append(tuple(k.split()))    
  return lst

In [ ]:
doc_vectors = []
for review in df.sentence:
  vectors = []
  doc_vectors.append(extract_bigrams(review))

In [ ]:
import itertools 

fdist = nltk.FreqDist(list(itertools.chain(*doc_vectors)))
common_bigrams = sorted(fdist.items(), key=lambda x: (-x[1]))

In [ ]:
lst_final = []
cmn_bigrams = [bigram for bigram in common_bigrams if bigram[1] > 1]
for bigram in cmn_bigrams:
  ctr = -1
  for lst in doc_vectors:
    ctr+=1
    if bigram[0] in lst:
      lst_final.append(extract_trigrams(df.sentence[ctr]))

In [ ]:
lst_words = []
for a,b,c in set(list(itertools.chain(*lst_final))):
  if ("VB" in pos_tag([a])[0][1]  or "VB" in pos_tag([b])[0][1] or "VB" in pos_tag([c])[0][1]) and not ("VB" in pos_tag([a])[0][1]  and "VB" in pos_tag([b])[0][1] and "VB" in pos_tag([c])[0][1]) and  (a in words.words() and b in words.words() and c in words.words()):
    phrase_list = list(nlp(" ".join([a,b,c])).noun_chunks)
    #print([a,b,c])
    print(phrase_list) if len(phrase_list) >= 1 and len(str(phrase_list[0]).split()) > 1 else print(" ".join([a,b,c]))

hat come half
access recipe save
[extremely slow problem]
[memory program]
search come fix
don know yahoo
[type need]
[bar slow crash]
buy night try
[slow process]
action painfully slow
time make search
[file photo]
[use icon]
wont work say
adjust moon happen
start irritate slow
time try save
consider book file
image disappear happen
add button selected
[column flicker]
[change password]
[exact video]
bug make impossible
[overwrite note]
[manage trip]
update make change
[preview word]
doesnt save favorite
doesnt save progress
[half hat]
update whats happen
review buy sound
update make harder
[brightness display]
save need fixed
[point work]
[new update]
update lose note
[happen month fix]
[map distance]
[file list]
game doesnt save
hat make want
work add button
[fix problem]
search slow time
io make legacy
io know bug
[far restaurant]
say update fix
phone like come
